# DNS-SEC Example

The following is an example of how DNS-SEC signs keys within the chain of trust. In this simple example:

1) We are using smaller prime numbers to speed up the calculations; normally, the prime numbers would be in the order of 100+ digits long. We are also using abbreviated zone data for the purpose of this exercise for the ikea.se domain. For full details fo the records, you can find zone data here: dnsviz.net/d/ikea.se/dnssec

2) We are using the same key values for zone signing keys (ZSK) and key siging keys (KSK); althought this is permitted under the DNSSEC protocol, it is generally best practice to use separate key values for the KSK and ZSK for security purposes.

3) We will be using RSA/SHA-256 for this exercise, which is the current DNSSEC standard

4) You will need to run SageMath Kernel to verify these calculations


## Zone Data

In [2]:
# Root Server data
Pr = 2**107 - 1
Qr = 2**607 - 1
Nr = Pr*Qr                    # Root modulus
phi_Nr = (Pr - 1)*(Qr - 1)
Er = 101                      # Root encryption exponent
Dr = inverse_mod(Er, phi_Nr)  # Root decryption exponent
root_data = "a.root-servers.net, b.root-servers.net, c.root-servers.net"

# ccTLD data, i.e. Sweden
Pcc = 2**521 - 1
Qcc = 2**127 - 1 
Ncc = Pcc*Qcc                    # ccTLD modulus
phi_Ncc = (Pcc - 1)*(Qcc - 1)
Ecc = 173                      # ccTLD encryption exponent
Dcc = inverse_mod(Ecc, phi_Ncc)  # ccTLD decryption exponent
ccTLD_data = "a.ns.se, b.ns.se, c.ns.se"

# Domain data
# domain data
# Base Values Bob
Pd = 49316606534387222628407057297693523964136820850511818985519515647415558322543
Qd = 91872478880438861140001013686985361958647305204999997818848240040038381660563
phi_Nd = (Pd - 1)*(Qd - 1)
Nd = Pd*Qd                    # domain modulus
Ed = 65537                      # domain encryption exponent
Dd = inverse_mod(Ed, phi_Nd)  # domain decryption exponent
domain_data = "udns1.cscdns.net, udns2.cscdns.uk"

## Zone Signing

In [4]:
def hash_data(zone_data):
    import hashlib                        
    hash = hashlib.sha256()               
    hash.update(zone_data.encode('UTF-8'))  
    return hash.hexdigest()               

ccTLD_hash = hash_data(ccTLD_data)
print("Hash of ccTLD zone data: " + ccTLD_hash + "\n")
domain_hash = hash_data(domain_data)
print("Hash of Ikea's domain data: " + domain_hash + "\n")

#Convert these hashes into integer values
ccTLD_hash_int = int(ccTLD_hash, 16)
domain_hash_int = int(domain_hash, 16)

#Here we will sign our hashes of the zone data with the private key of the parent zone

ccTLD_signature = power_mod(ccTLD_hash_int, Dr, Nr) #the country code is signed with the root zone's zone signing key
domain_signature = power_mod(domain_hash_int, Dcc, Ncc) #the ikea domian is signed with the ccTLD zone siging key

print("ccTLD Signature: " + str(ccTLD_signature) + "\n")
print("domain Signature: " + str(domain_signature) + "\n")

Hash of ccTLD zone data: 8bb943b39c114a350ee715de34ac8f8ac7c8a4bf6b16a79c8149a4732b7c28f8

Hash of Ikea's domain data: 512395a1d2aca6a50b09528654e0e3e614b1348adfd61a0b3c2798a8f9d82f1b

ccTLD Signature: 48326013451064070964245397333755131872164607716684758219186948759475540649148571974279647248798896414750756387222191981876795843626936515390239288818702354207928967178331433929056254858521931760522205859913918581072

domain Signature: 642115898653489816736130500756345885811892976654718248273637484652613806774066841992439062981662968044216182934120867449884001641918256074560348256036893248226159583671018204764519925187767009214



## Signature Verification

In [20]:
# Now we are going to pretend ot be a recursive resolver, and one of our clinets wants to visit Ikea.se.
# By obtaining the data and the signatures, all I need to do is check that they match.
# This check will assure me that the data has not been tampered with.

#Knowing the hash function is SHA-256, we verify it ourselves with the zone data
ccTLD_verify_hash = hash_data(ccTLD_data)
domain_verify_hash = hash_data(domain_data)

# We verify the signature by decrypting the received signature with the public key of the zone above it...
decrypted_ccTLD = hex(power_mod(ccTLD_signature, Er, Nr))[2:]
decrypted_domain = hex(power_mod(domain_signature, Ecc, Ncc))[2:]

#Verify that the two hash values match:
print("Do the values of the two hashes match?")

print("Is the provided ccTLD signature's hash the same as ours? " + str(decrypted_ccTLD==ccTLD_verify_hash))
print("Is the provided Ikea domain's hash the same as ours?" + str(decrypted_domain==domain_verify_hash))

#As these values match, we now know the values received are validated by the parent zone servers.


Do the values of the two hashes match?
ccTLD: True
Ikea domain: True


## References

The values of E, D, and N (composite of p and q), and the formula for converting between hashes and signatures, were taken from CY202 Cryptography Lab 1 -- part 2 as part of the UC Berkeley MICS curriculum. In this way we did not need to calculate independent values of E & D that were 1 mod N for the purpose of this demonstration, which is for DNSSEC.